# Regex NLP
- Sara Echeverría 21371

### Objetivo
Aplicar únicamente expresiones regulares con la librería re para eliminar información personal identificable de transcripciones médicas.

In [6]:
import pandas as pd
import re

In [7]:
filePath = "data/pii_dataset.csv"
df = pd.read_csv(filePath)

In [8]:
escapedNames = [re.escape(name) for name in df['Nombre'].tolist()]
namePattern = r'\b(?:' + '|'.join(escapedNames) + r')\b'

In [9]:
def cleanPiiFromText(text):
    text = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w{2,4}\b', '[REDACTED]', text)
    text = re.sub(r'(\+?\d{1,3}[\s-]?)?(\d{9}|\d{3}[\s-]?\d{3}[\s-]?\d{3})', '[REDACTED]', text)
    text = re.sub(r'\b[XYZ]?\d{7,8}[A-Z]\b', '[REDACTED]', text)
    text = re.sub(r'\bP\d{7,8}\b', '[REDACTED]', text)
    text = re.sub(r'\b(Calle|Avenida|Urbanización|Alameda|Vial|Piso)\b[^,.]*', '[REDACTED]', text)
    text = re.sub(namePattern, '[REDACTED]', text, flags=re.IGNORECASE)
    return text.strip()

In [10]:
df['TextoLimpio'] = df['Texto'].apply(cleanPiiFromText)

In [11]:
df[['Texto', 'TextoLimpio']].head()

,Texto,TextoLimpio
0,"Mi nombre es Reyes Pinilla Rosado, puedes cont...","Mi nombre es [REDACTED], puedes contactarme po..."
1,"Mi nombre es Evita Solsona Escobar, puedes con...","Mi nombre es [REDACTED], puedes contactarme po..."
2,"Mi nombre es Lorenza Mate Bayón, puedes contac...","Mi nombre es [REDACTED], puedes contactarme po..."
3,"Mi nombre es Renato González-Araujo, puedes co...","Mi nombre es [REDACTED], puedes contactarme po..."
4,"Mi nombre es Chus Alejandro Duarte Mayoral, pu...","Mi nombre es [REDACTED], puedes contactarme po..."


### Discusión
Al utilizar únicamente expresiones regulares, se encontraron limitaciones relacionadas con la flexibilidad y exactitud en la detección de ciertos tipos de PII, especialmente nombres o direcciones con formatos variados. Esto se debe a que los patrones deben definirse de forma precisa para evitar falsos positivos o dejar información sin anonimizar, lo cual representa un reto cuando los datos no siguen una estructura uniforme.

Como mejora, se recomienda complementar este proceso con técnicas de NLP o modelos de reconocimiento de entidades, que permiten identificar esta información con mayor eficacia en textos no estructurados. No resolver correctamente este problema podría comprometer la privacidad de las personas, especialmente en contextos sensibles como el de la salud, y conllevar consecuencias éticas y legales importantes.